In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.decomposition import PCA
from sklearn.manifold import TSNE
from sklearn.preprocessing import StandardScaler
import umap.umap_ as umap
import seaborn as sns
import torch
from scipy.spatial.distance import pdist, squareform

np.random.seed(42)
torch.manual_seed(42)

In [ ]:
def get_one_hot_feature(input_dim, class_idx):
    feature = torch.zeros(1, input_dim)
    part_size = input_dim // 20
    start_idx = class_idx * part_size
    end_idx = start_idx + part_size
    if end_idx + part_size > input_dim:
        end_idx = input_dim
    feature[0, start_idx:end_idx] = 1
    return feature

In [ ]:
def doPCA(X, y, num_clusters):
    pca = PCA(n_components=2)
    X_pca = pca.fit_transform(X)
    plt.figure(figsize=(10, 8))
    scatter = plt.scatter(X_pca[:, 0], X_pca[:, 1], c=y, cmap='tab20')
    plt.xlabel('PCA Component 1')
    plt.ylabel('PCA Component 2')
    plt.title(f'PCA of Dataset with {num_clusters} Clusters')
    plt.colorbar(scatter, ticks=range(num_clusters))
    plt.show()

def doTSNE(X, y, num_clusters):
    tsne = TSNE(n_components=2, perplexity=5)
    X_tsne = tsne.fit_transform(X)
    plt.figure(figsize=(10, 8))
    scatter = plt.scatter(X_tsne[:, 0], X_tsne[:, 1], c=y, cmap='tab20')
    plt.xlabel('t-SNE Component 1')
    plt.ylabel('t-SNE Component 2')
    plt.title(f't-SNE of Dataset with {num_clusters} Clusters')
    plt.colorbar(scatter, ticks=range(num_clusters))
    plt.show()

def doUMAP(X, y, n_neighbors=15, min_dist=0.1, num_clusters=20):
    umap_model = umap.UMAP(n_components=2, n_neighbors=n_neighbors, min_dist=min_dist, random_state=42)
    X_umap = umap_model.fit_transform(X)

    plt.figure(figsize=(10, 8))
    scatter = plt.scatter(X_umap[:, 0], X_umap[:, 1], c=y, cmap='tab20')
    plt.xlabel('UMAP Component 1')
    plt.ylabel('UMAP Component 2')
    plt.title(f'UMAP of Dataset with {num_clusters} Clusters (n_neighbors={n_neighbors}, min_dist={min_dist})')
    plt.colorbar(scatter, ticks=range(num_clusters))
    plt.show()

def doAvgDistance(X, y):
    for dimension_reduction in ['PCA', 't-SNE', 'UMAP']:
        if dimension_reduction == 'PCA':
            pca = PCA(n_components=2)
            X_transformed = pca.fit_transform(X)
        elif dimension_reduction == 't-SNE':
            tsne = TSNE(n_components=2, perplexity=5)
            X_transformed = tsne.fit_transform(X)
        elif dimension_reduction == 'UMAP':
            umap_model = umap.UMAP(n_components=2, n_neighbors=15, min_dist=0.1, random_state=42)
            X_transformed = umap_model.fit_transform(X)

        unique_labels = np.unique(y)
        cluster_avg_distances = {}

        for label in unique_labels:
            cluster_points = X_transformed[np.array(y) == label]
            distances = pdist(cluster_points, metric='euclidean')
            avg_distance = np.mean(distances)
            cluster_avg_distances[label] = avg_distance

        total_distance = 0
        for label, avg_distance in cluster_avg_distances.items():
            total_distance += avg_distance

        print(f'Average distance between points in {dimension_reduction} space: {total_distance / len(cluster_avg_distances)}')

In [ ]:
############################################### 
###     one hot
###############################################

# num_clusters = 20
# samples_per_cluster = 100
# num_features = 20

# features = []
# labels = []

# for i in range(num_clusters):
#     for _ in range(samples_per_cluster):
#         one_hot = get_one_hot_feature(num_features, i)
#         features.append(one_hot)
#     labels.extend([i] * samples_per_cluster)

############################################### 
###     concat one hot
###############################################

# dim1 = 5
# dim2 = 5
# num_clusters = dim1 * dim2
# samples_per_cluster = 100
# num_features = 20

# features = []
# labels = []

# for i in range(dim1):
#     for j in range(dim2):
#         for _ in range(samples_per_cluster):
#             one_hot1 = get_one_hot_feature(num_features, i)
#             one_hot2 = get_one_hot_feature(num_features, j)
#             one_hot = torch.cat((one_hot1, one_hot2), dim=1)
#             features.append(one_hot)
#         labels.extend([i * dim1 + j] * samples_per_cluster)

###############################################
###     untargeted
###############################################

# num_clusters = 3
# features = []
# labels = []

# cache = torch.load("/home/jialin/data/cache/Meta-Llama-3.1-8B-Instruct/remove_untargeted.pt")
# features.extend([v for k, v in cache['known'].items()])
# labels.extend([0] * len(cache['known']))

# cache = torch.load("/home/jialin/data/cache/Meta-Llama-3.1-8B-Instruct/misclassify_untargeted.pt")
# features.extend([v for k, v in cache['known'].items()])
# labels.extend([1] * len(cache['known']))

# cache = torch.load("/home/jialin/data/cache/Meta-Llama-3.1-8B-Instruct/generate_untargeted.pt")
# features.extend([v for k, v in cache['known'].items()])
# labels.extend([2] * len(cache['known']))

###############################################
###     class name
###############################################

# voc_classes=['aeroplane', 'bicycle', 'bird', 'boat', 'bottle', 'bus', 'car', 'cat', 'chair', 'cow', 'diningtable', 'dog', 'horse', 'motorbike', 'person', 'pottedplant', 'sheep', 'sofa', 'train', 'tvmonitor']
# num_clusters = 20
# cache = torch.load("cache.pt")
# features = []
# labels = []

# for i, cls in enumerate(voc_classes):
#     for _ in range(100):
#         features.extend(cache[cls].unsqueeze(0))
#         labels.extend([i] * len(cache[cls]))

###############################################
###    feature TARGETED REMOVE
###############################################

# voc_classes=['aeroplane', 'bicycle', 'bird', 'boat', 'bottle', 'bus', 'car', 'cat', 'chair', 'cow', 'diningtable', 'dog', 'horse', 'motorbike', 'person', 'pottedplant', 'sheep', 'sofa', 'train', 'tvmonitor']
# num_clusters = 20
# cache = torch.load("/home/jialin/data/cache/Meta-Llama-3.1-8B-Instruct/remove_targeted.pt")
# features = []
# labels = []

# for i, cls in enumerate(voc_classes[:num_clusters]):
#     features.extend([v for k, v in cache['known'][cls].items()])
#     # labels.extend([i] * len(cache['known'][cls])) # every class is a cluster
#     labels.extend([j for j in range(len(cache['known'][cls]))]) # every template is a cluster

###############################################
###    feature TARGETED REMOVE with instruction
###############################################

voc_classes=['aeroplane', 'bicycle', 'bird', 'boat', 'bottle', 'bus', 'car', 'cat', 'chair', 'cow', 'diningtable', 'dog', 'horse', 'motorbike', 'person', 'pottedplant', 'sheep', 'sofa', 'train', 'tvmonitor']
num_clusters = 20
# cache = torch.load("/home/jialin/mmdetection/mmdet/AnywhereDoor/Meta-Llama-3.1-8B-Instruct/remove_targeted_Auto.pt")
# cache = torch.load("/home/jialin/mmdetection/mmdet/AnywhereDoor/Meta-Llama-3.1-8B-Instruct/remove_targeted_AutoCausal.pt")
# cache = torch.load("/home/jialin/mmdetection/mmdet/AnywhereDoor/Meta-Llama-3.1-8B/remove_targeted_Auto.pt")
# cache = torch.load("/home/jialin/mmdetection/mmdet/AnywhereDoor/Meta-Llama-3.1-8B/remove_targeted_AutoCausal.pt")
# cache = torch.load("/home/jialin/mmdetection/mmdet/AnywhereDoor/Llama-2-7b-chat-hf/remove_targeted_Auto.pt")
# cache = torch.load("/home/jialin/mmdetection/mmdet/AnywhereDoor/Llama-2-7b-chat-hf/remove_targeted_AutoCausal.pt")
# cache = torch.load("/home/jialin/mmdetection/mmdet/AnywhereDoor/Llama-2-7b-chat-hf/remove_targeted_Llama.pt")
cache = torch.load("/home/jialin/mmdetection/mmdet/AnywhereDoor/Llama-2-7b-chat-hf/remove_targeted_LlamaCausal.pt")
features = []
labels = []

for i, cls in enumerate(voc_classes[:num_clusters]):
    features.extend([v for k, v in cache['known'][cls].items()])
    labels.extend([i] * len(cache['known'][cls])) # every class is a cluster
    # labels.extend([j for j in range(len(cache['known'][cls]))]) # every template is a cluster

In [ ]:
features = [f.to(torch.float32) for f in features]
X = torch.cat(features, dim=0).cpu().numpy()
y = np.array(labels)

scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)
doPCA(X_scaled, y, num_clusters)
doTSNE(X_scaled, y, num_clusters)
doUMAP(X_scaled, y, num_clusters=num_clusters)
doAvgDistance(X_scaled, y)